In [ ]:
#Install the following depedencies first
#nltk.download('punkt')
#nltk.download('stopwords')
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove*.zip

In [ ]:
#Importing Pandas Library for reading article.csv fie
import pandas as pd

In [ ]:
#Reading the article.csv file
df = pd.read_csv("articlee.csv",encoding = 'unicode_escape')

In [ ]:
#Input_Text
df['article_text'][0]

In [ ]:
#Tokenization
import nltk
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

In [ ]:
#first 5 sentence
sentences[:5]

In [ ]:
# Extract word vectors
import numpy as np
word_embeddings = {}
f = open('glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [ ]:
len(word_embeddings)

In [ ]:
#removing punctuations, numbers and special words
import pandas as pd
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

In [ ]:
# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [ ]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [ ]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]

In [ ]:
clean_sentences[:5]

In [ ]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((100,))
  sentence_vectors.append(v)

In [ ]:
sentence_vectors[0].shape

In [ ]:
#similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [ ]:
sim_mat[0][1]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

In [ ]:
nx_graph

In [ ]:
print(nx_graph)

In [ ]:
type(scores)

In [ ]:
print(len(scores))

In [ ]:
scores

In [ ]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

In [ ]:
type(ranked_sentences)

In [ ]:
y = len(ranked_sentences)
print(y)

In [ ]:
# Extract top 5 sentences as the summary
for i in range(5):
  print(ranked_sentences[i][1])
  print()